<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/Trafilatura_crawling_032525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trafilatura: 쉬운 웹 텍스트 스크래핑


## trafilatura
- 이탈리아어로 '압출' 또는 '추출' 의미
- 웹 크롤링을 쉽고 효율적으로 수행할 수 있게 해주는 Python 라이브러리.
- 웹 페이지에서 중요한 텍스트 콘텐츠(본문, 제목, 메타데이터)를 다양한 형식으로 쉽게 추출.

## 주요 특징
1. **간결한 코드**: 몇 줄의 코드만으로 HTML에서 핵심 콘텐츠 추출
2. **자동 콘텐츠 감지**: 웹 페이지의 구조를 분석하여 중요한 내용을 자동으로 구분
3. **다양한 출력 형식**: txt, XML, JSON, CSV, MARKDOWN 등 다양한 형식으로 결과 출력
4. **메타데이터 추출**: 제목, 작성자, 날짜, 카테고리 등의 메타데이터도 함께 추출
5. **다국어 지원**: 한국어를 포함한 다양한 언어로 된 웹 페이지 처리

## 설치

In [ ]:
# trafilatura 설치
!pip install trafilatura

In [ ]:
# 필요한 라이브러리 임포트
import trafilatura
import pandas as pd
import matplotlib.pyplot as plt
import json
from IPython.display import HTML, display
import time

## 1. 기본 사용법

trafilatura의 가장 기본적인 사용 방법은 두 단계로 이루어짐

1. `fetch_url()` 함수로 웹 페이지 다운로드
2. `extract()` 함수로 웹 페이지에서 콘텐츠 추출


### 단일 URL에서 텍스트 추출

In [ ]:
# 웹 페이지 다운로드 및 콘텐츠 추출
url = 'https://n.news.naver.com/mnews/article/079/0004008336'
downloaded = trafilatura.fetch_url(url)

# 기본 텍스트 추출
result = trafilatura.extract(downloaded)

# 결과 출력 (앞부분 1000자만)
print("URL:", url)
print("추출된 텍스트 (앞부분):", result[:1000] if result else "콘텐츠 추출 실패")
print("추출된 텍스트 길이:", len(result) if result else 0, "자")

### 메타데이터 추출

In [ ]:
# 메타데이터와 함께 추출
'''메타데이터_변수 = ['title', 'author', 'hostname', 'date', 'fingerprint', 'id',
'license', 'comments', 'raw_text', 'text', 'language', 'image', 'pagetype',
'filedate', 'source', 'source-hostname', 'excerpt', 'categories', 'tags']'''

result_with_metadata = trafilatura.extract(downloaded, output_format='json', with_metadata=True)

# JSON 문자열을 파이썬 사전으로 변환
if result_with_metadata:
    metadata_dict = json.loads(result_with_metadata)

# 데이터프레임으로 출력
pd.DataFrame([metadata_dict])

# 깔끔하게 출력
print("제목:", metadata_dict.get('title', '정보 없음'))
print("작성자:", metadata_dict.get('author', '정보 없음'))
print("날짜:", metadata_dict.get('date', '정보 없음'))
print("설명:", metadata_dict.get('description', '정보 없음'))
print("언어:", metadata_dict.get('language', '정보 없음'))
print("카테고리:", metadata_dict.get('categories', '정보 없음'))
print("태그:", metadata_dict.get('tags', '정보 없음'))
print("사이트 이름:", metadata_dict.get('sitename', '정보 없음'))
print("\n본문 내용 (앞부분):", metadata_dict.get('text', '')[:500] + '...' if metadata_dict.get('text') else "콘텐츠 없음")

## 2. 다양한 출력 형식

trafilatura는 다양한 출력 형식을 지원:

- **텍스트(txt)**: 기본 형식, 일반 텍스트로 콘텐츠 추출
- **JSON**: 구조화된 JSON 형식으로 메타데이터와 본문 제공
- **XML**: XML 형식으로 문서 구조 유지
- **CSV**: 테이블 형식으로 데이터 저장에 유용
- **MARKDOWN**: 마크다운 형식으로 제목, 링크 등의 구조 유지

출력 형식은 `extract()` 함수의 `output_format` 매개변수로 지정.

### 출력 형식 예제

In [ ]:
# 다양한 출력 형식 비교
formats = ['txt', 'json', 'xml', 'csv', 'markdown']
results = {}

for fmt in formats:
    result = trafilatura.extract(downloaded, output_format=fmt)
    results[fmt] = result[:500] + '...' if result and len(result) > 500 else result

# 결과 출력
for fmt, result in results.items():
    print(f"\n===== {fmt.upper()} 형식 =====")
    print(result)

## 3. 고급 설정

다양한 옵션을 통해 웹 페이지 추출 방식을 세밀하게 제어

- **include_comments**: 댓글 포함 여부 (기본값: True)
- **include_tables**: 테이블 포함 여부 (기본값: True)
- **include_images**: 이미지 참조 포함 여부 (기본값: True)
- **include_links**: 링크 포함 여부 (기본값: True)
- **no_fallback**: 메인 알고리즘 실패 시 대체 알고리즘 사용 여부 (기본값: False)
- **favor_precision**: 정확도 우선(True) 또는 재현율 우선(False) (기본값: False)
- **target_language**: 특정 언어로 된 콘텐츠만 추출 (예: 'ko', 'en')

### 고급 설정 예제

In [ ]:
# 다양한 설정 옵션 사용 예제
# 1. 모든 옵션 켜기
full_option = trafilatura.extract(
    downloaded,
    include_comments=True,
    include_tables=True,
    include_images=True,
    include_links=True,
    output_format='txt'
)

# 2. 최소 옵션 (순수 텍스트만)
minimal_option = trafilatura.extract(
    downloaded,
    include_comments=False,
    include_tables=False,
    include_images=False,
    include_links=False,
    output_format='txt'
)

# 결과 비교
print("== 모든 옵션 켜기 (전체 콘텐츠) ==")
print(full_option[:500], "...\n") if full_option else print("추출 실패\n")

print("== 최소 옵션 (순수 텍스트만) ==")
print(minimal_option[:500], "...") if minimal_option else print("추출 실패")

# 텍스트 길이 비교
if full_option and minimal_option:
    print(f"\n전체 콘텐츠 길이: {len(full_option)} 자")
    print(f"순수 텍스트 길이: {len(minimal_option)} 자")
    print(f"차이: {len(full_option) - len(minimal_option)} 자")

## 4. 여러 URL 처리

여러 웹 페이지를 한 번에 크롤링해야 하는 경우, 여러 URL을 효율적으로 처리

- 반복문 사용: `trafilatura.fetch_url()`와 `trafilatura.extract()`를 반복문에서 순차적으로 사용
- concurrent.futures 모듈 사용: Python의 표준 라이브러리인 concurrent.futures를 사용하여 병렬 처리
- trafilatura.crawl 모듈 사용: 현재 작동 어려움


In [ ]:
# 테스트할 URL 목록
urls = [
    "https://www.domin.co.kr/news/articleView.html?idxno=1500714",  # 지역지
    "https://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0003099071&CMPT_CD=P0010&utm_source=naver&utm_medium=newsearch&utm_campaign=naver_news",  # 인터넷언론
    "https://www.sisajournal-e.com/news/articleView.html?idxno=218656",  # 잡지
    "https://news.tvchosun.com/site/data/html_dir/2025/02/03/2025020390247.html?_gl=1*60n6cu*_ga*MTY2MDQ5OTkwNy4xNzM4NTgxMTYw*_ga_D5GZR50LJV*MTczODYzODM5MS4yLjEuMTczODYzODQxOC4zMy4wLjA.",  # 방송1
    "https://news.sbs.co.kr/news/endPage.do?news_id=N1007969124&plink=THUMB&cooper=SBSNEWSPROGRAM",  # 방송2
    "https://www.yna.co.kr/view/AKR20250130040200053?input=1195m",  # 통신사
    "https://m.skyedaily.com/news_view.html?ID=256827"  # 추출 안되는 경우
]

# 1. 기본 순차 처리: 반복문 사용
print("1. 순차 처리 방식")
start_time = time.time()
results_sequential = []

for url in urls:
    try:
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            result = trafilatura.extract(downloaded, output_format='json', with_metadata=True)
            if result:
                results_sequential.append(json.loads(result))
            else:
                print(f"콘텐츠 추출 실패: {url}")
        else:
            print(f"다운로드 실패: {url}")
    except Exception as e:
        print(f"오류 발생: {url} - {e}")

sequential_time = time.time() - start_time
print(f"순차 처리 시간: {sequential_time:.2f}초")
print(f"성공적으로 처리된 URL: {len(results_sequential)}/{len(urls)}\n")
print(results_sequential)

# 2. concurrent.futures를 사용한 병렬 처리
import concurrent.futures

def process_url(url):
    try:
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            result = trafilatura.extract(downloaded, output_format='json', with_metadata=True)
            if result:
                return json.loads(result)
        return None
    except Exception:
        return None

print("2. 병렬 처리 방식")
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results_parallel = list(filter(None, executor.map(process_url, urls)))

parallel_time = time.time() - start_time
print(f"병렬 처리 시간: {parallel_time:.2f}초")
print(f"성공적으로 처리된 URL: {len(results_parallel)}/{len(urls)}")
print(f"속도 향상: {sequential_time/parallel_time:.2f}배\n")
print(results_parallel)

## 5. trafilatura와 다른 라이브러리 비교

### 5.1. 주요 라이브러리 비교

| 라이브러리 | 주요 특징 | 장점 | 단점 |
|----------|---------|------|------|
| **trafilatura** | 콘텐츠 추출 특화 | 간결한 코드, 자동 콘텐츠 감지, 다양한 출력 형식 | HTML 구조 조작에 제한적 |
| **BeautifulSoup** | HTML/XML 파싱 | 세밀한 HTML 조작, 유연성 | 콘텐츠 추출에 추가 로직 필요 |
| **Scrapy** | 대규모 크롤링 | 완전한 크롤링 프레임워크, 분산 크롤링 | 학습 곡선이 높음, 설정이 복잡 |
| **newspaper3k** | 뉴스 기사 특화 | 뉴스 전용 기능, NLP 지원 | 뉴스 외 콘텐츠에 제한적 |
| **requests-html** | 최신 파싱 라이브러리 | JavaScript 실행 지원, 간결한 API | 성능이 상대적으로 낮을 수 있음 |

### 5.2. 사용 시나리오별 권장 라이브러리

- **뉴스 기사 크롤링**: trafilatura, newspaper3k
- **복잡한 웹 구조 파싱**: BeautifulSoup, lxml
- **대규모 웹 크롤링**: Scrapy
- **JavaScript 렌더링 필요**: Selenium, Playwright, requests-html
- **빠른 프로토타이핑**: trafilatura, requests-html

trafilatura는 특히 블로그, 뉴스 기사, 포럼 등의 텍스트 중심 웹사이트에서 콘텐츠를 추출하는 데 우수함.